In [3]:
import skvideo.io
from c3d import C3D
from keras.models import Model
from sports1M_utils import preprocess_input, decode_predictions

Using TensorFlow backend.


In [4]:
base_model = C3D(weights='sports1M')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc6').output)

In [71]:
base_model.save("c3d.h5")

In [65]:
vid_path = 'test2/'
vid = skvideo.io.vread(vid_path)
length = vid.shape[0]
video = vid[int((length*2)/5):int((length*4)/5)]
x = preprocess_input(vid)

In [43]:
print(video.shape)

(56, 360, 640, 3)


In [5]:
print(int((length*2)/5))
print(int((length*4)/5))
print(length)

573
1147
1434


In [7]:
from PIL import Image

In [62]:
from IPython.display import Video
Video(vid_path)


In [37]:
import cv2 
  
# Function to extract frames 
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    # Used as counter variable 
    count = 0  
    # checks whether frames were extracted 
    success = 1  
    while success:   
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read()   
        # Saves the frames with frame-count
        if count == int(length/2):
            cv2.imwrite("extracted/framed.jpg", image)   
        count += 1

In [66]:
FrameCapture(vid_path)

In [67]:
features = model.predict(x)

In [11]:
import os
l = os.listdir('')

FileNotFoundError: [Errno 2] No such file or directory: ''

In [41]:
lbls = {}
lbls[0] = 'Explosion'
lbls[1] = 'Arrest'
lbls[2] = 'Assault'
lbls[3] = 'Burglary'
lbls[4] = 'Abuse'
lbls[5] = 'Fighting'
lbls[6] = 'normal'
lbls[7] = 'Arson'


In [13]:
import pickle
filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [68]:
prediction_ans = loaded_model.predict(features)

print(prediction_ans[0])
print(lbls[prediction_ans[0]])
print(prediction_ans)

6
normal
[6]


In [15]:
def upper(imagePath):
    from google.cloud import storage
    from firebase import firebase
    import os
    import random
    nm = random.randint(1,100000)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="sihproject-e8cc7-7fb9245f560e.json"
    firebase = firebase.FirebaseApplication('https://sihproject-e8cc7.firebaseio.com//')
    client = storage.Client()
    bucket = client.get_bucket('sihproject-e8cc7.appspot.com')
# posting to firebase storage
    imageBlob = bucket.blob("/")
# imagePath = [os.path.join(self.path,f) for f in os.listdir(self.path)]
    imageBlob = bucket.blob(str(nm))
    imageBlob.upload_from_filename(imagePath)
    url_img = imageBlob.public_url
    return url_img

In [16]:
def poster(url_img):
    import json
    import requests
    import time
    postData= dict()
    postData["camera_id"]='ABC10'
    postData["activity_name"]= lbls[prediction_ans[0]]
    postData["screenshot_url"]=url_img
    postData["start_time"]="2020-01-20T10:00:28"
    postData["end_time"]="2020-01-20T10:00:28"
    headers = {'content-type': 'application/json'}
    r = requests.post('https://safevision.herokuapp.com/api/activity/', data=json.dumps(postData),headers=headers)
    if r.status_code == 200:
        print("Request sent")
    else:
        print(r.status_code)
    

In [63]:
from IPython.display import Video
Video(vid_path)


In [69]:
prediction_ans = loaded_model.predict(features)

print(prediction_ans[0])
print("Predicted:" + lbls[prediction_ans[0]])
print(prediction_ans)

6
Predicted:normal
[6]


In [70]:
if prediction_ans[0] != 6:
    path = upper("extracted/framed.jpg")
    poster(path)

In [18]:
print(path)

https://storage.googleapis.com/sihproject-e8cc7.appspot.com/59699


In [62]:
import numpy as np
data_x = np.load('datax.npy')
data_y = np.load('datay.npy')

print(data_x.shape)
print(data_y.shape)



(550, 4096)
(550,)


In [63]:
from sklearn.svm import LinearSVC

In [64]:
svm = LinearSVC()
svm.fit(data_x,data_y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
preds = svm.predict(data_x)

In [15]:
import pickle

In [16]:
filename = 'finalized_model.sav'
pickle.dump(svm, open(filename, 'wb'))